# Train and Apply Models

In [1]:
from ML.model_training import (
    omit_patient_video,
    single_user_split,
    train_lstm,
    train_random_forest,
    build_lstm_sequences
)
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
)
import numpy as np
import pandas as pd
from ML import utils
import sys
import random
from itertools import product

In [2]:
# remove_list = [5, 18]
remove_list = []

def balance(X, y, seed=5):
    c = y.value_counts()
    if c.get("high", 0) == c.get("low", 0):
        return X.reset_index(drop=True), y.reset_index(drop=True)
    maj = c.idxmax()
    m = c.min()
    keep = y[y != maj].index.union(y[y == maj].sample(m, random_state=seed).index)
    return X.loc[keep].reset_index(drop=True), y.loc[keep].reset_index(drop=True)

# subjects = [i for i in range(0, 23)]
subjects = [3]

## LSTM CV Optimizer

This is currently running LOSO CV, by removing 18 videos (all) from each user.

In [16]:
param_grid = {
    "lr": [0.0001],
    "epochs": [100],
    "units": [128],
    "batch_size": [64],
    "timesteps": [128]
}

best_params = None
best_mean_acc = -float("inf")

print("Starting global hyperparameter search...\n")

for lr, epochs, units, batch_size, timesteps in product(
    param_grid["lr"],
    param_grid["epochs"],
    param_grid["units"],
    param_grid["batch_size"],
    param_grid["timesteps"],
):
    combo_accs = []
    for i in subjects:
        relabel = False
        while True:
            while True:
                X_train_df, X_test_df, arousal_train, arousal_test = omit_patient_video(
                    target="arousal",
                    selected_user=i,
                    trials=18,
                    # holdout_videos=[2, 10, 15],
                    exclude_users=remove_list,
                    filename="datasets/features_table.csv",
                    relabel=relabel,
                )

                features = utils.filter_features(
                    X_train_df.columns,
                    remove_bands=["gamma"],
                )
                features = [c for c in features if c in X_train_df.columns]
                features.remove("arousal")
                features.remove("valence")
                features.remove("patient_index")
                features.remove("video_index")

                # Build sequence-level data (trials = (patient, video))
                X_train_seq, y_train_seq = build_lstm_sequences(
                    X_train_df,
                    features,
                    target_col="arousal",
                    thresh=3.8,
                    fixed_T=timesteps,
                )
                X_test_seq, y_test_seq = build_lstm_sequences(
                    X_test_df,
                    features,
                    target_col="arousal",
                    thresh=3.8,
                    fixed_T=timesteps
                )

                minority = (y_train_seq == 1.0).sum() < (y_train_seq == 0.0).sum()
                maj_label = 0.0 if minority else 1.0
                min_label = 1.0 - maj_label

                idx_maj = np.where(y_train_seq == maj_label)[0]
                idx_min = np.where(y_train_seq == min_label)[0]

                if len(idx_min) == 0:
                    # no samples of one class, redo split
                    print("No minority class in this split, re-drawing...")
                    continue

                # reps = int(np.ceil(len(idx_maj) / len(idx_min)))
                # idx_min_upsampled = np.tile(idx_min, reps)[: len(idx_maj)]

                # idx_balanced = np.concatenate([idx_maj, idx_min_upsampled])
                # np.random.shuffle(idx_balanced)

                # X_train_bal = X_train_seq[idx_balanced]
                # y_train_bal = y_train_seq[idx_balanced]

                print("y_train counts:", np.bincount(y_train_seq.astype(int)))
                print("y_test counts:", np.bincount(y_test_seq.astype(int)))

                break

            lstm, X_test_eval, y_test_eval = train_lstm(
                X_train_seq,
                X_test_seq,
                y_train_seq,
                y_test_seq,
                lr=lr,
                epochs=epochs,
                units=units,
                batch_size=batch_size,
                dropout=0.4,
                recurrent_dropout=0.2,
                bidirectional=True,
            )
            y_prob = lstm.predict(X_test_eval).ravel()
            arousal_pred = (y_prob >= 0.5).astype(int)

            acc = accuracy_score(y_test_eval, arousal_pred)
            print(acc)
            relabel = False
            break

        combo_accs.append(float(acc))

        print("\nConfusion Matrix (subject):")
        print(confusion_matrix(y_test_eval, arousal_pred))
    mean_acc = float(np.mean(combo_accs))
    print(
        f"Params lr={lr}, epochs={epochs}, units={units}, batch_size={batch_size}, timesteps={timesteps} "
        f"-> mean acc across subjects = {mean_acc:.4f}"
    )

    if mean_acc > best_mean_acc:
        best_lstm = lstm
        best_mean_acc = mean_acc
        best_params = {
            "lr": lr,
            "epochs": epochs,
            "units": units,
            "batch_size": batch_size,
            "timesteps": timesteps
        }


print("\nBest universal hyperparameters:")
print(best_params)
print(f"Mean accuracy across subjects (tuning): {best_mean_acc:.4f}\n")

Starting global hyperparameter search...

Held-out patient: 3 | Held-out (patient, video) trials: [(3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17)] | Excluded users: []
y_train counts: [220 176]
y_test counts: [13  5]
X_train_arr shape: (396, 128, 476)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step
0.7222222222222222

Confusion Matrix (subject):
[[13  0]
 [ 5  0]]
Params lr=0.0001, epochs=100, units=128, batch_size=64, timesteps=128 -> mean acc across subjects = 0.7222

Best universal hyperparameters:
{'lr': 0.0001, 'epochs': 100, 'units': 128, 'batch_size': 64, 'timesteps': 128}
Mean accuracy across subjects (tuning): 0.7222



### Fine-tune for User

In [25]:
from tensorflow.keras.models import clone_model
from tensorflow.keras import optimizers
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf
import numpy as np

param_grid = {
    "lr": [0.0001],
    "epochs": [25],
    "units": [256],
    "batch_size": [128],
    "patience": [8],
}

N_RUNS = 18
SELECTED_USER = 3

best_params = None
best_mean_acc = -np.inf

print("Starting per-user fine-tuning hyperparameter search...\n")

for lr, epochs, units, batch_size, patience in product(
    param_grid["lr"],
    param_grid["epochs"],
    param_grid["units"],
    param_grid["batch_size"],
    param_grid["patience"],
):

    combo_accs = []

    print(
        f"Testing combo: lr={lr}, batch_size={batch_size}, "
        f"epochs={epochs}, patience={patience}"
    )

    for run in range(N_RUNS):
        (
            X_user_train_df,
            X_user_test_df,
            arousal_user_train,
            arousal_user_test,
        ) = single_user_split(
            target="arousal",
            selected_user=SELECTED_USER,
            holdout_videos=[run],
            k_holdouts=1,
            filename="datasets/features_table.csv",
        )

        X_user_train_seq, y_user_train_seq = build_lstm_sequences(
            X_user_train_df,
            feature_cols=features,
            target_col="arousal",
            thresh=3.8,
            fixed_T=128,
        )
        X_user_test_seq, y_user_test_seq = build_lstm_sequences(
            X_user_test_df,
            feature_cols=features,
            target_col="arousal",
            thresh=3.8,
            fixed_T=128,
        )

        user_model = clone_model(best_lstm)
        user_model.build(best_lstm.input_shape)
        user_model.set_weights(best_lstm.get_weights())

        # Freeze all but last layer
        # for layer in user_model.layers[:-1]:
        #     layer.trainable = False

        user_model.compile(
            optimizer=optimizers.Adam(learning_rate=lr),
            loss="binary_crossentropy",
            metrics=["accuracy"],
        )

        history = user_model.fit(
            X_user_train_seq,
            y_user_train_seq,
            validation_split=0.1,
            verbose=False,
            batch_size=batch_size,
            epochs=epochs,
            callbacks=[
                tf.keras.callbacks.EarlyStopping(
                    patience=patience,
                    restore_best_weights=True,
                    monitor="val_loss",
                )
            ],
        )

        test_loss, test_acc = user_model.evaluate(
            X_user_test_seq, y_user_test_seq, verbose=0
        )
        combo_accs.append(test_acc)

    mean_acc = np.mean(combo_accs)
    std_acc = np.std(combo_accs)
    print(f"  -> mean acc over {N_RUNS} runs: {mean_acc:.4f} " f"(std={std_acc:.4f})\n")

    if mean_acc > best_mean_acc:
        best_mean_acc = mean_acc
        best_params = {
            "lr": lr,
            "batch_size": batch_size,
            "epochs": epochs,
            "units": units,
            "patience": patience,
        }

print("\nBest hyperparameters for user", SELECTED_USER)
print(best_params)
print(f"Best mean test accuracy: {best_mean_acc:.4f}\n")


# Final evaluation with best hyperparams
acc_list = []
all_cm = np.zeros((2, 2), dtype=int)

print("Running final evaluation with best hyperparameters...\n")

for run in range(N_RUNS):
    (
        X_user_train_df,
        X_user_test_df,
        arousal_user_train,
        arousal_user_test,
    ) = single_user_split(
        target="arousal",
        selected_user=SELECTED_USER,
        k_holdouts=1,
        filename="datasets/features_table.csv",
    )

    X_user_train_seq, y_user_train_seq = build_lstm_sequences(
        X_user_train_df,
        feature_cols=features,
        target_col="arousal",
        thresh=3.8,
        fixed_T=128,
    )
    X_user_test_seq, y_user_test_seq = build_lstm_sequences(
        X_user_test_df,
        feature_cols=features,
        target_col="arousal",
        thresh=3.8,
        fixed_T=128,
    )

    print(f"Run {run}")
    print("User train seq shape:", X_user_train_seq.shape)
    print("User test  seq shape:", X_user_test_seq.shape)
    print("y_train counts:", np.bincount(y_user_train_seq.astype(int)))
    print("y_test  counts:", np.bincount(y_user_test_seq.astype(int)))

    user_model = clone_model(best_lstm)
    user_model.build(best_lstm.input_shape)
    user_model.set_weights(best_lstm.get_weights())

    for layer in user_model.layers[:-1]:
        layer.trainable = False

    user_model.compile(
        optimizer=optimizers.Adam(learning_rate=best_params["lr"]),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    history = user_model.fit(
        X_user_train_seq,
        y_user_train_seq,
        validation_split=0.1,
        verbose=False,
        batch_size=best_params["batch_size"],
        epochs=best_params["epochs"],
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                patience=best_params["patience"],
                restore_best_weights=True,
                monitor="val_accuracy",
            )
        ],
    )

    test_loss, test_acc = user_model.evaluate(
        X_user_test_seq, y_user_test_seq, verbose=0
    )
    print(f"\nRun {run} - User Test accuracy: {test_acc:.4f}")

    acc_list.append(test_acc)

    # Predictions & confusion matrix
    y_prob = user_model.predict(X_user_test_seq, verbose=0).ravel()
    y_pred = (y_prob >= 0.5).astype("int32")

    cm = confusion_matrix(y_user_test_seq.astype("int32"), y_pred, labels=[0, 1])
    all_cm += cm

    print("\nConfusion Matrix (rows=true, cols=pred):")
    print(cm)
    print(
        "\nOverall accuracy:", accuracy_score(y_user_test_seq.astype("int32"), y_pred)
    )
    print("-" * 50)

acc_array = np.array(acc_list)
print(f"\nMean user test accuracy over {len(acc_array)} runs: {acc_array.mean():.4f}")
print(f"Std of user test accuracy over {len(acc_array)} runs: {acc_array.std():.4f}")

print("\nPooled confusion matrix over all runs:")
print(all_cm)

Starting per-user fine-tuning hyperparameter search...

Testing combo: lr=0.0001, batch_size=128, epochs=25, patience=8


KeyboardInterrupt: 